# Analog Devices FX2LP test
- [firmware](https://ez.analog.com/rf/f/discussions/75850/adf4350-and-adf4351-software-and-usb-microcontroller-firmware-source-codes#52311)

### 需安裝

```
pip install pyusb

```

### Windows DLL
[libusb-win32-devel-filter-1.2.6.0.exe](https://sourceforge.net/projects/libusb-win32/files/libusb-win32-releases/1.2.6.0/)  

https://sourceforge.net/projects/libusb-win32/  



### Linux 權限
https://ubuntuforums.org/showthread.php?t=1682084  
https://linuxconfig.org/tutorial-on-how-to-write-basic-udev-rules-in-linux  
```
If anyone is interested.. fixed with udev rule:

cd /etc/udev/rules.d/

nano 70-ch341.rules

SUBSYSTEM=="usb", ATTRS{idVendor}=="1a86", GROUP="dialout"
SUBSYSTEM=="usb", ATTRS{idVendor}=="1a86", MODE="666"

then reloaded udev rules:

$ sudo udevadm control --reload-rules  
$ sudo udevadm trigger

```

In [1]:
%pylab inline  

import usb
from array import array

Populating the interactive namespace from numpy and matplotlib


## Backend

## Device

In [2]:
# import usb.backend.libusb1 as libusb
import usb.backend.libusb0 as libusb 

device = usb.core.find(idVendor = 0X0456, idProduct = 0XB40D, backend = libusb.get_backend())
device.set_configuration()

## Data

In [3]:
def to_bytes_array(reg_value): 
    bytes_array = array('B',  reg_value.to_bytes(length = 4, byteorder = 'big')) 
    bytes_array.insert(0, 8 * len(bytes_array))
    return  bytes_array

def send(bytes_array):
    return device.ctrl_transfer(bmRequestType = 0x40, bRequest = 0xDD, wValue = 0, wIndex = 0,
                                data_or_wLength = bytes_array[::-1]) 

In [4]:
INITIAL_REGISTERS_VALUES = [0x3c0020, 0x80087d1, 0x30041c2, 0xe404b3, 0x932224, 0x580005]

for reg_value in INITIAL_REGISTERS_VALUES[::-1]:
    bytes_array = to_bytes_array(reg_value)
    send(bytes_array)